In [12]:
import pickle
import os.path
import datetime
import re
from pprint import pprint
import pandas as pd

In [13]:
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from httplib2 import Http
import base64, html2text
from IPython.display import clear_output

In [14]:
with open('token.pickle', 'rb') as token:
    credentials = pickle.load(token)
service = build('gmail','v1',credentials=credentials)
results = service.users().labels().list(userId='me').execute()
results

{'labels': [{'id': 'CHAT',
   'name': 'CHAT',
   'messageListVisibility': 'hide',
   'labelListVisibility': 'labelHide',
   'type': 'system'},
  {'id': 'SENT', 'name': 'SENT', 'type': 'system'},
  {'id': 'INBOX',
   'name': 'INBOX',
   'messageListVisibility': 'hide',
   'labelListVisibility': 'labelShow',
   'type': 'system'},
  {'id': 'IMPORTANT', 'name': 'IMPORTANT', 'type': 'system'},
  {'id': 'TRASH',
   'name': 'TRASH',
   'messageListVisibility': 'hide',
   'labelListVisibility': 'labelHide',
   'type': 'system'},
  {'id': 'DRAFT',
   'name': 'DRAFT',
   'messageListVisibility': 'hide',
   'labelListVisibility': 'labelShow',
   'type': 'system'},
  {'id': 'SPAM',
   'name': 'SPAM',
   'messageListVisibility': 'hide',
   'labelListVisibility': 'labelHide',
   'type': 'system'},
  {'id': 'CATEGORY_FORUMS', 'name': 'CATEGORY_FORUMS', 'type': 'system'},
  {'id': 'CATEGORY_UPDATES', 'name': 'CATEGORY_UPDATES', 'type': 'system'},
  {'id': 'CATEGORY_PERSONAL', 'name': 'CATEGORY_PERSONA

In [39]:
results = service.users().messages().list(
    userId='me',
    q='jio recharge successful <phone_number>').execute()

In [42]:
results['messages'][:5]

[{'id': '1781c369de7cad84', 'threadId': '1781c369de7cad84'},
 {'id': '177f2e9db8a51319', 'threadId': '177f2e9db8a51319'},
 {'id': '177de6f5352a6bad', 'threadId': '177de6f5352a6bad'},
 {'id': '1776e619514f631e', 'threadId': '1776e619514f631e'},
 {'id': '1776e6193386d0d4', 'threadId': '1776e6193386d0d4'}]

In [17]:
messages = results.get('messages')
i=0
recharges=[]
for msg in messages:
    i += 1
    message = service.users().messages().get(
        userId='me',id=msg['id'],format='full').execute()
    m=message['payload']
    d=message['internalDate']
    recharges.append((m,d))
    print('Number of emails parsed = ',i,flush=True)
    clear_output(wait=True)
print('Number of emails parsed = ',i,'\nDone')

Number of emails parsed =  83 
Done


In [18]:
with open('recharge.pickle', 'wb') as f:
    pickle.dump(recharges,f)

In [19]:
with open('recharge.pickle', 'rb') as f: 
    recharges = pickle.load(f)

In [23]:
def email_decoder(m):
    """
    Converts an html formatted b64encoded string into
    utf8 string.
    """
    b = base64.urlsafe_b64decode(m)
    t = html2text.html2text(b.decode('utf-8'))
    return t

def sensible_date2str(x: datetime.date) -> str:
    """
    Returns string of the form:
    '15-Dec-2012'
    """
    return x.strftime("%d-%b-%Y")

def ms2ctime(t):
    """
    Converts unix time (time since 1st January 1970 UTC) in 
    milliseconds into ctime.
    """
    t=int(t)/1000
    t = datetime.datetime.fromtimestamp(int(t))
    return sensible_date2str(t)

def regexer(string):
    pattern = re.compile(r"""
    (?<=Rs\. ).*?(?= is)| # (negative look "Rs. ")(match)(positive look " is")
    (?<=mber ).*?(?=\.)|  # phone number
    (?<=)\d*?(?=[ ]?GB )| # total data
    ...\W?\wB(?=\/Day)   # ...(match any 3)(match 0or1 [space])(match 'B')
    """, re.VERBOSE)
    return re.findall(pattern, string)

def get_recursively(search_dict, field):
    """
    Takes a dict with nested lists and dicts,
    and searches all dicts for a key of the field
    provided.
    """
    fields_found = []
    for key, value in search_dict.items():
        if key == field:
            fields_found.append(value)
        elif isinstance(value, dict):
            results = get_recursively(value, field)
            for result in results:
                fields_found.append(result)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    more_results = get_recursively(item, field)
                    for another_result in more_results:
                        fields_found.append(another_result)
    return fields_found

In [24]:
rechrgs = []
for i in range(len(recharges)):
    x = get_recursively(recharges[i][0], 'data')
    y = email_decoder(x[0])
    z = regexer(y)
    _date = ms2ctime(recharges[i][1])
    try:
        d = dict(date=_date,rupees=z[0], data_GB=z[2], per_day=z[-1])
    except: continue
    rechrgs.append(d)

In [30]:
pprint(rechrgs[:5], indent=6, sort_dicts=False)

[     {     'date': '10-Mar-2021',
            'rupees': ' 555.00 ',
            'data_GB': '126',
            'per_day': '1.5 GB'},
      {     'date': '02-Mar-2021',
            'rupees': ' 251.00 ',
            'data_GB': '50',
            'per_day': ''},
      {     'date': '26-Feb-2021',
            'rupees': ' 21.00 ',
            'data_GB': '2',
            'per_day': ''},
      {     'date': '05-Feb-2021',
            'rupees': ' 251.00 ',
            'data_GB': '50',
            'per_day': ''},
      {     'date': '16-Jan-2021',
            'rupees': ' 251.00 ',
            'data_GB': '50',
            'per_day': ''}]


In [26]:
df = pd.DataFrame(rechrgs)
df.head()

,date,rupees,data_GB,per_day
0,10-Mar-2021,555.00,126,1.5 GB
1,02-Mar-2021,251.00,50,
2,26-Feb-2021,21.00,2,
3,05-Feb-2021,251.00,50,
4,16-Jan-2021,251.00,50,


In [27]:
def diff_days(date_string):
    diff = datetime.datetime.today() - datetime.datetime.strptime(date_string,'%d-%b-%Y')
    return diff.days

In [28]:
df['Days_since'] = df['date'].apply(diff_days)

In [29]:
df.head()

,date,rupees,data_GB,per_day,Days_since
0,10-Mar-2021,555.00,126,1.5 GB,3
1,02-Mar-2021,251.00,50,,11
2,26-Feb-2021,21.00,2,,15
3,05-Feb-2021,251.00,50,,36
4,16-Jan-2021,251.00,50,,56
